# Recurrent Bernoulli Variational Autoencoder for Energy Disaggregation
## (a.k.a. variational BOLT)
The corresponding paper is under review at the moment. But here is the code for anyone who wants to play with it.
It's completely unsupervised and all you would have to do is feed data into it.

Important: This code runs only with keras 2.0.1. Because InputSpecs were changes in later versions, it won't run with a newer version of keras than 2.0.1.

In order to speed up computations, in a pre-processing step, events are extracted from aggregate power.

In [8]:
from scipy import cluster
import pickle
import numpy as np
import keras.backend as K
from keras.layers import Input, LSTM, TimeDistributed, activations, constraints, initializers, regularizers, Recurrent
from keras.layers.recurrent import _time_distributed_dense
from keras.legacy import interfaces
from keras.optimizers import RMSprop
from keras.models import Model
from keras.constraints import Constraint
from keras.layers.core import Dense
from keras.engine import InputSpec
import tensorflow as tf
import keras

import itertools

'''
Input: data is a matrix of shape (t,s) containing aggregate instantaneous power waveforms
            [s being the number of samples per voltage-cycle, t number of cycles]
            
Output: A dictionary containing mean inter-event waveforms and their respective lengths

'''
def extract_events(data):
    active_power = np.mean(data,axis=1)
    delta = np.abs(active_power[1:] - active_power[:-1])
    
    w = 5
    delta[delta<50] = 0
    
    votes = np.zeros(delta.shape)
    for i in range(len(data)):
        low = np.max([0,i-w])
        high = np.min([i+w,len(delta)])
        window = delta[low:high]
        best = np.argmax(window)
        if delta[low+best] > 0:
            votes[low+best]+=1
    
    ev = [i for i,x in enumerate(votes) if x>3]
    ev.append(len(data))
    
    low = 0
    snippets_agg = []
    snippets_length = []
    
    for e in ev:
        snippets_length.append(e-low)
        snippets_agg.append(np.mean(data[low+1:e],axis=0))
        low = e
        
    ''' Uncomment this if you want to see a plot of what's going on
    plt.plot(np.mean(data,axis=-1))
    plt.plot(votes*100)
    for e in ev:
        plt.axvline(x=e, color='r')
    '''
        
    return {'snippets_agg':np.array(snippets_agg), 'snippets_len':np.array(snippets_length)}

The following is a slight modification of the original $keras$ implementation of the LSTM. The only difference is that it outputs at time $t$ are $o(t)$ and $o(t-1)$. This allows for a much cleaner implementation of the variational loss, i.e. we don't have to deal with corner cases in the temporal mini-batch.

In [2]:
class LSTM_tm1(Recurrent):
    """Long-Short Term Memory unit - Hochreiter 1997.
    For a step-by-step description of the algorithm, see
    [this tutorial](http://deeplearning.net/tutorial/lstm.html).
    # Arguments
        units: Positive integer, dimensionality of the output space.
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you don't specify anything, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        recurrent_activation: Activation function to use
            for the recurrent step
            (see [activations](../activations.md)).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs.
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix,
            used for the linear transformation of the recurrent state.
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        unit_forget_bias: Boolean.
            If True, add 1 to the bias of the forget gate at initialization.
            Setting it to true will also force `bias_initializer="zeros"`.
            This is recommended in [Jozefowicz et al.](http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf)
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
    # References
        - [Long short-term memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf) (original 1997 paper)
        - [Learning to forget: Continual prediction with LSTM](http://www.mitpressjournals.org/doi/pdf/10.1162/089976600300015015)
        - [Supervised sequence labeling with recurrent neural networks](http://www.cs.toronto.edu/~graves/preprint.pdf)
        - [A Theoretically Grounded Application of Dropout in Recurrent Neural Networks](http://arxiv.org/abs/1512.05287)
    """
    @interfaces.legacy_recurrent_support
    def __init__(self, units,
                 activation='hard_sigmoid',
                 recurrent_activation='hard_sigmoid',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 unit_forget_bias=True,
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 **kwargs):
        super(LSTM_tm1, self).__init__(**kwargs)
        self.units = units
        self.activation = activations.get(activation)
        self.recurrent_activation = activations.get(recurrent_activation)
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.unit_forget_bias = unit_forget_bias

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(recurrent_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = min(1., max(0., dropout))
        self.recurrent_dropout = min(1., max(0., recurrent_dropout))

    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape = input_shape[0]

        batch_size = input_shape[0] if self.stateful else None
        self.input_dim = input_shape[2]
        self.input_spec = InputSpec(shape=(batch_size, None, self.input_dim))
        self.state_spec = [InputSpec(shape=(batch_size, self.units)),
                           InputSpec(shape=(batch_size, self.units))]

        self.states = [None, None]
        if self.stateful:
            self.reset_states()

        self.kernel = self.add_weight((self.input_dim, self.units * 4),
                                      name='kernel',
                                      initializer=self.kernel_initializer,
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        self.recurrent_kernel = self.add_weight(
            (self.units, self.units * 4),
            name='recurrent_kernel',
            initializer=self.recurrent_initializer,
            regularizer=self.recurrent_regularizer,
            constraint=self.recurrent_constraint)

        if self.use_bias:
            self.bias = self.add_weight((self.units * 4,),
                                        name='bias',
                                        initializer=self.bias_initializer,
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
            if self.unit_forget_bias:
                bias_value = np.zeros((self.units * 4,))
                bias_value[self.units: self.units * 2] = 1.
                K.set_value(self.bias, bias_value)
        else:
            self.bias = None

        self.kernel_i = self.kernel[:, :self.units]
        self.kernel_f = self.kernel[:, self.units: self.units * 2]
        self.kernel_c = self.kernel[:, self.units * 2: self.units * 3]
        self.kernel_o = self.kernel[:, self.units * 3:]

        self.recurrent_kernel_i = self.recurrent_kernel[:, :self.units]
        self.recurrent_kernel_f = self.recurrent_kernel[:, self.units: self.units * 2]
        self.recurrent_kernel_c = self.recurrent_kernel[:, self.units * 2: self.units * 3]
        self.recurrent_kernel_o = self.recurrent_kernel[:, self.units * 3:]

        if self.use_bias:
            self.bias_i = self.bias[:self.units]
            self.bias_f = self.bias[self.units: self.units * 2]
            self.bias_c = self.bias[self.units * 2: self.units * 3]
            self.bias_o = self.bias[self.units * 3:]
        else:
            self.bias_i = None
            self.bias_f = None
            self.bias_c = None
            self.bias_o = None
        self.built = True

    def preprocess_input(self, inputs, training=None):
        if self.implementation == 0:
            input_shape = K.int_shape(inputs)
            input_dim = input_shape[2]
            timesteps = input_shape[1]

            x_i = _time_distributed_dense(inputs, self.kernel_i, self.bias_i,
                                          self.dropout, input_dim, self.units,
                                          timesteps, training=training)
            x_f = _time_distributed_dense(inputs, self.kernel_f, self.bias_f,
                                          self.dropout, input_dim, self.units,
                                          timesteps, training=training)
            x_c = _time_distributed_dense(inputs, self.kernel_c, self.bias_c,
                                          self.dropout, input_dim, self.units,
                                          timesteps, training=training)
            x_o = _time_distributed_dense(inputs, self.kernel_o, self.bias_o,
                                          self.dropout, input_dim, self.units,
                                          timesteps, training=training)
            return K.concatenate([x_i, x_f, x_c, x_o], axis=2)
        else:
            return inputs

    def get_constants(self, inputs, training=None):
        constants = []
        if self.implementation == 0 and 0 < self.dropout < 1:
            input_shape = K.int_shape(inputs)
            input_dim = input_shape[-1]
            ones = K.ones_like(K.reshape(inputs[:, 0, 0], (-1, 1)))
            ones = K.tile(ones, (1, int(input_dim)))

            def dropped_inputs():
                return K.dropout(ones, self.dropout)

            dp_mask = [K.in_train_phase(dropped_inputs,
                                        ones,
                                        training=training) for _ in range(4)]
            constants.append(dp_mask)
        else:
            constants.append([K.cast_to_floatx(1.) for _ in range(4)])

        if 0 < self.recurrent_dropout < 1:
            ones = K.ones_like(K.reshape(inputs[:, 0, 0], (-1, 1)))
            ones = K.tile(ones, (1, self.units))

            def dropped_inputs():
                return K.dropout(ones, self.recurrent_dropout)
            rec_dp_mask = [K.in_train_phase(dropped_inputs,
                                            ones,
                                            training=training) for _ in range(4)]
            constants.append(rec_dp_mask)
        else:
            constants.append([K.cast_to_floatx(1.) for _ in range(4)])
        return constants

    def step(self, inputs, states):
        h_tm1 = states[0]
        c_tm1 = states[1]
        dp_mask = states[2]
        rec_dp_mask = states[3]

        if self.implementation == 2:
            z = K.dot(inputs * dp_mask[0], self.kernel)
            z += K.dot(h_tm1 * rec_dp_mask[0], self.recurrent_kernel)
            if self.use_bias:
                z = K.bias_add(z, self.bias)

            z0 = z[:, :self.units]
            z1 = z[:, self.units: 2 * self.units]
            z2 = z[:, 2 * self.units: 3 * self.units]
            z3 = z[:, 3 * self.units:]

            i = self.recurrent_activation(z0)
            f = self.recurrent_activation(z1)
            c = f * c_tm1 + i * self.activation(z2)
            o = self.activation(z3)
        else:
            if self.implementation == 0:
                x_i = inputs[:, :self.units]
                x_f = inputs[:, self.units: 2 * self.units]
                x_c = inputs[:, 2 * self.units: 3 * self.units]
                x_o = inputs[:, 3 * self.units:]
            elif self.implementation == 1:
                x_i = K.dot(inputs * dp_mask[0], self.kernel_i) + self.bias_i
                x_f = K.dot(inputs * dp_mask[1], self.kernel_f) + self.bias_f
                x_c = K.dot(inputs * dp_mask[2], self.kernel_c) + self.bias_c
                x_o = K.dot(inputs * dp_mask[3], self.kernel_o) + self.bias_o
            else:
                raise ValueError('Unknown `implementation` mode.')

            i = self.recurrent_activation(x_i + K.dot(h_tm1 * rec_dp_mask[0],
                                                      self.recurrent_kernel_i))
            f = self.recurrent_activation(x_f + K.dot(h_tm1 * rec_dp_mask[1],
                                                      self.recurrent_kernel_f))
            c = f * c_tm1 + i * self.activation(x_c + K.dot(h_tm1 * rec_dp_mask[2],
                                                            self.recurrent_kernel_c))
            o = self.activation(x_o + K.dot(h_tm1 * rec_dp_mask[3],
                                                      self.recurrent_kernel_o))
        h = o * self.activation(c)
        if 0 < self.dropout + self.recurrent_dropout:
            h._uses_learning_phase = True
            
        h_out = K.concatenate([h, h_tm1],axis=-1)
        return h_out, [h, c]

    def get_config(self):
        config = {'units': self.units,
                  'activation': activations.serialize(self.activation),
                  'recurrent_activation': activations.serialize(self.recurrent_activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer': initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'unit_forget_bias': self.unit_forget_bias,
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer': regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer': regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint': constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout}
        base_config = super(LSTM_tm1, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

The following is a constraint that states that the instantaneous power of all inferred components must be strictly positive.

In [3]:
class pinst_plus(Constraint):
    
    def __call__(self,p):
        pinst = K.mean(p,axis=1)
        pinst_offset = K.clip(pinst,-1000,0)
        p = K.transpose(p)
        p -= pinst_offset
        p = K.transpose(p)
        
        return p

## Variational loss function
The following code is where 'the magic happens'.

Possible research paths:
- Using a different distance measurement other than Euclidean distance for the difference loss would done by changing $fn\_onerror$ or $fn\_offerror$ and $noerror$
- if you wanted to disaggregate 20+ components, then you would need to drop the sparsity constraint and create the matrix Z by sampling from probs. I have code somewhere that generates the k most probable configurations according to q, i.e. probs. Contact me.

In [4]:
class var_loss:
    
    def __init__(self, layer, num_comps, time_delay, output_dim):
        self.layer = layer #the dummy layer containing the component waveforms
        self.num_comps = num_comps #number of inferred components
        self.time_delay = time_delay #size of the minibatch
        self.output_dim = output_dim #dimensionality of the aggregate and difference
        
        self.__name__ = 'var_loss'

        
    def __call__(self, y_true, y_pred):

        W = self.layer.layer.kernel  #component waveforms

        dy_true = y_true[:,:,self.output_dim:] #desired diffference
        y_true = y_true[:,:,:self.output_dim]  #desired aggregate

        #all probabilities are clipped between EPS and 1-EPS to avoid log(0)
        EPS = 0.00001

        y_pred = K.clip(y_pred,EPS, 1-EPS)
        probs = y_pred[:,:,0:self.num_comps] #sigma(t), i.e. component activation probabilities in Q
        probs_tm1 = y_pred[:,:,self.num_comps:] #sigma(t-1)

        
        Z, Z_np = self.all_subsets(5, self.num_comps) #Z constitutes all admissable sparse configurations
        
        print('Building network')
        
        error_func = K.square #if you change this to K.abs, you will approximately use Laplace instead of Gaussians
        zw = K.dot(Z,W) #the waveforms of all sparse configurations
        
        probs_tm1_nograd = tf.stop_gradient(probs_tm1) #for when the gradient shouldnt flow
        
        #next bloc computes losses associated with every component switching on or off at every time step
        #in the temporal batch, as well as no component switching, i.e. O(t,i), I(t,i) and X(t,i) from paper
        fn_onerror = lambda wi: -K.sum(error_func(dy_true - wi),axis=-1)
        fn_offerror = lambda wi: -K.sum(error_func(dy_true + wi),axis=-1)
        onerrors = tf.transpose(tf.map_fn(fn_onerror, W, parallel_iterations=1000),[1,2,0])
        offerrors = tf.transpose(tf.map_fn(fn_offerror, W, parallel_iterations=1000),[1,2,0])
        noerrors = -K.sum(error_func(dy_true),axis=-1)
        
        #in the following three tensors are computed: they are all of the shape [batch_size, t, num_z]
        #num_z is the number of admissable sparse configurations
        fn_rmse = lambda x: -K.sum(error_func(y_true - x),axis=-1)
        fn_prob = lambda x: K.prod(probs*x + (1-probs)*(1-x),axis=-1)
        fn_diff_tm1 = lambda z: K.sum(probs_tm1_nograd*(1-z)*offerrors + (1-probs_tm1_nograd)*z*onerrors,axis=-1) + \
                                K.prod(probs_tm1_nograd*z + (1-probs_tm1_nograd)*(1-z),axis=-1)*noerrors

        alpha = 1.0
        beta = 1.0
        #these are the 3 matrices
        Ls = tf.transpose(tf.map_fn(fn_rmse, zw, parallel_iterations=1000),[1,2,0]) #aggregate loss for every z
        Ps = tf.transpose(tf.map_fn(fn_prob, Z, parallel_iterations=1000),[1,2,0]) #q(z|x) for every z
        Ds = tf.transpose(tf.map_fn(fn_diff_tm1, Z, parallel_iterations=1000),[1,2,0]) #difference loss for every z

        Ls = (Ds)*beta+Ls*alpha #combine losses
        
        #now we feed the loss into a softmax to obtain p(z|x)
        e = K.exp(Ls - K.max(Ls, axis=-1, keepdims=True))
        s = K.sum(e, axis=-1, keepdims=True)
        Ls_sm = tf.stop_gradient(e/s) #p(z|x)
        
        y = K.dot(Ls_sm,Z) #this computes E[z] w.r.t. p(z|x) - simple dot product
        E_pxz = K.sum(tf.stop_gradient(Ps)*Ls,axis=-1) #E[p(x,z)] w.r.t. q(z|x)
        return (-K.sum(y*K.log(probs) + (1-y)*K.log(1-probs)) -E_pxz)/1000.0 #done


    def all_subsets(self, num_act, num_c):
        vals = np.eye(num_c)
        Z = [np.zeros((num_c,))]
        
        for a in range(1,num_act+1):
            for c in itertools.combinations(vals,a):
                Z.append(np.sum(c,axis=0))
                
        return tf.constant(np.array(Z,dtype=np.float32)), np.array(Z,dtype=np.float32)

## Helper functions
In the following the network topology is defined, as well as some helper function that help training the network

In [9]:
'''
This creates the actual neural network. If you want to change the network topology. This is where you do it.
'''

def get_net(input_dim, num_comps, time_delay, output_dim):
    
    inp = Input(batch_shape=(batch_size,time_delay,input_dim), name='y_input')

    lx = TimeDistributed(Dense(250, activation = 'relu'))(inp)
    lz = TimeDistributed(Dense(250, activation = 'relu'))(lx)
    bn = keras.layers.BatchNormalization()(lz)
    ly = TimeDistributed(Dense(150, activation = 'tanh'))(bn)
    
    dummy_layer = TimeDistributed(Dense(num_comps), name='dummy')(ly)
    recon = TimeDistributed(Dense(int(output_dim), activation = 'linear', W_constraint=pinst_plus()), name='waveforms')
    reconed = recon(dummy_layer)
    
    l0 = LSTM(100, activation='tanh', return_sequences = True, consume_less='gpu', stateful=True)(ly)
    l2 = LSTM_tm1(num_comps, activation='sigmoid', return_sequences = True, stateful=True)(l0)
    
    
    model = Model(input=inp, output=[reconed, l2])
    optimizer = RMSprop(lr=0.0001)
    
    model.compile(optimizer=optimizer,loss=['mse', var_loss(recon, num_comps, time_delay, output_dim)],
                                            loss_weights = [0., 1.], sample_weight_mode='temporal')
    
    return model

'''
Use this to train the network. It resets internal states of the network when needed
It also takes the length of the different snippets into account (sample weights)
Note that all weights are saved after each iteration
'''
import time
def reset_train(net, input, output, sample_weights, t_batch, num_comps):
    weights = []
    errors = []
    weights.append(net.get_weights())
    sample_weights = np.expand_dims(sample_weights[1:],0)
    
    for i in range(200):
        try:
            try:
                init_states = np.zeros((1,num_comps))
                init_states[0,0] = 1
                net.layers[-1].reset_states([init_states,init_states])
                errors_in_batch = []
                start = time.time()
                for j in range(int(input.shape[1]/t_batch)):
                    io = input[:,j*t_batch:(j+1)*t_batch,:]
                    oo = [y[:,j*t_batch:(j+1)*t_batch,:] for y in output]
                    sw = sample_weights[:,j*t_batch:(j+1)*t_batch]
                    h = net.train_on_batch(io, oo, sample_weight=[np.ones((1,20)),sw])
                    errors_in_batch.append(h[0])
                    print('batch done', h[0])
                errors.append(np.mean(errors_in_batch))
                weights.append(net.get_weights())
                print(errors)
                print('done in', np.round(start - time.time()))
                np.save('/tmp/weights_ongoing_np',weights)
                np.save('/tmp/error_ongoing_np',errors)
            except KeyboardInterrupt:
                print('out')
                break
        except Exception as e:
            print('I messed up:',e)
    return weights, errors


'''
Use this to predict component activations once the network is trained
'''
def reset_pred(net, input, t_batch, num_comps):
    init_states = np.zeros((1,num_comps))
    init_states[0,0] = 1
    net.layers[-1].reset_states([init_states,init_states])
    #net.reset_states()
    outputs_in_batch = []
    for j in range(int(input.shape[1]/t_batch)):
        io = input[:,j*t_batch:(j+1)*t_batch,:]
        h = net.predict_on_batch(io)
        outputs_in_batch.append(h)
        print('batch done')

    probs = np.concatenate([x[1] for x in outputs_in_batch],axis=1)
    return probs



'''
prepare_data converts the snippets extracted into the required format
Also makes the inputs zero-mean unit-variance so we don't
'''
def prepare_data(d):
    
    a = np.zeros((d.shape[0]-1, 2*d.shape[1]),dtype=np.float32)
    b = np.zeros((d.shape[0]-1, 2*d.shape[1]),dtype=np.float32)
    
    l = d.shape[1]
    
    for i in range(len(d)-1):
        a[i,:l] = d[i,:]
        a[i,l:] = d[i+1,:] - d[i,:]
        b[i,:l] = d[i+1,:]
        b[i,l:] = d[i+1,:] - d[i,:]
    
    a = a-np.mean(a,axis=0)
    a = a/np.sqrt(np.var(a,axis=0)+0.001)
    
    return a,b

        

time_delay = 20
batch_size = 1

'''
Performs K-means clustering on the difference signal in order to initialize component waveforms
'''
def wf_init(b, num_comps, input_dim):
    threshold = 10
    diffs = np.mean(b[0,:,input_dim:],axis=-1)>threshold
    c, _ = cluster.vq.kmeans(b[0,diffs,input_dim:], num_comps)
    return c

'''
I recommend to pickle the dictionary create by the event detection algorithm.
Here the pickle file is loaded and the data is brought into the correct format.
'''

def load_snippets():
    d = pickle.load(open('redd31-events.pkl','rb'))
    dat = d['snippets_agg']
    a,b = prepare_data(dat)
    a = np.expand_dims(a,axis=0)
    b = np.expand_dims(b,axis=0)
    
    return a,b,d['snippets_len']

'''
Self explanatory: data is loaded, network created and then trained.
'''
def run():
    input_dim = 275
    input_scale = 500.0
    num_comps = 10
    a,b,l = load_snippets()
    print('creating net')
    print('net created')
    wf0 = wf_init(b,num_comps, input_dim)
    n = get_net(2*input_dim, num_comps, time_delay, input_dim)
    w = n.get_weights()
    w[15] = wf0/input_scale
    n.set_weights(w)
    print('starting to train')
    w,e = reset_train(n, a, [b[:,:,:input_dim]/input_scale, 
                             b/input_scale], l, time_delay, num_comps)
    
run()

creating net
net created


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(275, kernel_constraint=<__main__...., activation="linear")`
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, stateful=True, activation="tanh", return_sequences=True, implementation=2)`


TypeError: 'InputSpec' object does not support indexing